<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#回归和分类的区别" data-toc-modified-id="回归和分类的区别-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>回归和分类的区别</a></span></li><li><span><a href="#logistic-(逻辑斯蒂)-回归" data-toc-modified-id="logistic-(逻辑斯蒂)-回归-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>logistic (逻辑斯蒂) 回归</a></span></li><li><span><a href="#softmax-回归" data-toc-modified-id="softmax-回归-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>softmax 回归</a></span><ul class="toc-item"><li><span><a href="#softmax-相关函数对比" data-toc-modified-id="softmax-相关函数对比-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>softmax 相关函数对比</a></span></li></ul></li></ul></div>

## 回归和分类的区别

给定一些数据，{（x1，y1），（x2，y2）…（xn，yn） }，用 x 的值来预测 y 的值，通常地，y 的值是连续的就是回归问题，y 的值是离散的就叫分类问题

## logistic (逻辑斯蒂) 回归

logistic 回归模型公式：
![logistic](illustration/logistic.png)

logistic 回归与线性回归的区别：
* 线性回归中，X各维度叠加和就是Y，也就是Y与X就是线性相关
* logistic回归中，X各维度叠加和（或X各维度）与Y不是线性关系，而是一种平滑的变化

> logistic 回归用于二分类问题（y 只有两个值A,B，也可以写成1和0），回归模型得到的结果不是预测样本 X 对应的 y 值

## softmax 回归

Softmax 回归是一个线性的多类分类模型，实际上它是直接从 logistic 回归模型转化而来的。区别在与 Logistic 回归模型为二分类模型，而Softmax模型为多类分类模型

Softmax 函数的主要功能是将各个类别的"打分"转化成合理的(0-1)之间的概率值，并且加起来正好等于 1

> `tf.nn.sigmoid_cross_entropy_with_logits` 中的 labels 每一维可以包含多个`1`;  譬如： 一张待测图片中可以同时包含多个检测对象 "猫" 和 "狗"

> `tf.nn.softmax_cross_entropy_with_logits` 中的 labels 每一维只能包含一个`1` (数据必须经过 One-Hot Encoding 编码)；譬如：一张待测图片中只能包含一个检测对象 "猫" 或 "狗"

> `tf.nn.sparse_softmax_cross_entropy_with_logits` 是 tf.nn.softmax_cross_entropy_with_logits 的简易版本，除了输入参数不同，作用和算法实现是一样的。(该函数不要求传入的的 labels 必须经过 one-hot encoding 处理)

### softmax 相关函数对比

```python
tf.nn.softmax_cross_entropy_with_logits(
    _sentinel=None,
    labels=None,
    logits=None,
    dim=-1,
    name=None
)  # (记为 f1)

tf.nn.softmax_cross_entropy_with_logits_v2 (
    _sentinel=None,
    labels=None,
    logits=None,
    dim=-1,
    name=None
)  # (记为 f2)

tf.nn.sparse_softmax_cross_entropy_with_logits(
    _sentinel=None,
    labels=None,
    logits=None,
    name=None
)  # (记为 f3)
```

f1 和 f3 对于参数 `logits` 的要求都是一样的，即未经处理的，直接由神经网络输出的数值， 比如 [3.5, 2.1, 7.89, 4.4]。两个函数不一样的地方在于 `labels` 格式的要求，f1 的要求`labels`的格式和`logits`类似，比如[0, 0, 1, 0]。而 f3 的要求 `labels`是一个数值，这个数值记录着ground truth所在的索引。以[0, 0, 1, 0]为例，这里真值1的索引为2。所以 f3 要求 `labels` 的输入为数字 2(tensor)。一般可以用 `tf.argmax()` 来从[0, 0, 1, 0]中取得真值的索引。

<br/>f1 和 f2之间很像，实际上官方文档已经标记出 f1 已经是`deprecated`状态，推荐使用 f2。两者唯一的区别在于 f1 在进行反向传播的时候，只对`logits`进行反向传播，`labels`保持不变。而 f2 在进行反向传播的时候，同时对`logits`和`labels`都进行反向传播，如果将`labels`传入的 tensor 设置为 stop_gradients，就和 f1 一样了。

```python
import tensorflow as tf
import numpy as np

Truth = np.array([0, 0, 1, 0])
Pred_logits = np.array([3.5, 2.1, 7.89, 4.4])

loss1 = tf.nn.softmax_cross_entropy_with_logits(labels=Truth, logits=Pred_logits)
loss2 = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Truth, logits=Pred_logits)
loss3 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.argmax(Truth), logits=Pred_logits)

with tf.Session() as sess:
    print sess.run(loss1) 
    print sess.run(loss2)
    print sess.run(loss3)
```